In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import os.path
import torch.nn.functional as F
from torch.nn import init
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import random
from time import time
from sklearn.metrics import mean_squared_error

In [2]:
full_train_data = pd.read_csv('./data/full_train_set.csv')
train_data = pd.read_csv('./data/train_set.csv')
val_data = pd.read_csv('./data/val_set.csv')
test_data = pd.read_csv('./data/test_set.csv')

In [3]:
train_data.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'year', 'month',
       'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors',
       'median_visitors', 'max_visitors', 'count_observations',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'air_genre_name0', 'air_area_name0', 'air_genre_name1',
       'air_area_name1', 'air_genre_name2', 'air_area_name2',
       'air_genre_name3', 'air_area_name3', 'air_genre_name4',
       'air_area_name4', 'air_genre_name5', 'air_area_name5',
       'air_genre_name6', 'air_area_name6', 'air_genre_name7',
       'air_area_name7', 'air_genre_name8', 'air_area_name8',
       'air_genre_name9', 'air_area_name9', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
       'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int',
       'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2', 'date',
       'dayofquarter', 'quarter', 'day', 'dayofyear', 

In [4]:
set(train_data.air_store_id)

{'air_00a91d42b08b08d9',
 'air_0164b9927d20bcc3',
 'air_0241aa3964b7f861',
 'air_0328696196e46f18',
 'air_034a3d5b40d5b1b1',
 'air_036d4f1ee7285390',
 'air_0382c794b73b51ad',
 'air_03963426c9312048',
 'air_04341b588bde96cd',
 'air_049f6d5b402a31b2',
 'air_04cae7c1bc9b2a0b',
 'air_0585011fa179bcce',
 'air_05c325d315cc17f5',
 'air_0647f17b4dc041c8',
 'air_064e203265ee5753',
 'air_066f0221b8a4d533',
 'air_06f95ac5c33aca10',
 'air_0728814bd98f7367',
 'air_0768ab3910f7967f',
 'air_07b314d83059c4d2',
 'air_07bb665f9cdfbdfb',
 'air_082908692355165e',
 'air_083ddc520ea47e1e',
 'air_0845d8395f30c6bb',
 'air_084d98859256acf0',
 'air_0867f7bebad6a649',
 'air_08ba8cd01b3ba010',
 'air_08cb3c4ee6cd6a22',
 'air_08ef81d5b7a0d13f',
 'air_08f994758a1e76d4',
 'air_09040f6df960ddb8',
 'air_0919d54f0c9a24b8',
 'air_09661c0f3259cc04',
 'air_09a845d5b5944b01',
 'air_09fd1f5c58583141',
 'air_0a74a5408a0b8642',
 'air_0b184ec04c741a6a',
 'air_0b1e72d2d4422b20',
 'air_0b9038300f8b2b50',
 'air_0e1eae99b8723bc1',


In [18]:
full_data_train = full_train_data[['id','air_store_id', 'visit_date',
                                   'holiday_flg','dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear',
                                   'min_visitors', 'mean_visitors',
                                  'median_visitors', 'max_visitors', 'count_observations',
                                  'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
                                  'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 
                                  'total_reserv_sum','total_reserv_mean', 'total_reserv_dt_diff_mean', 
                                   'date_int','var_max_lat', 'var_max_long', 'lon_plus_lat','visitors']]
data_train = train_data[['id','air_store_id', 'visit_date',
                                   'holiday_flg','dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear',
                                   'min_visitors', 'mean_visitors',
                                  'median_visitors', 'max_visitors', 'count_observations',
                                  'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
                                  'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 
                                  'total_reserv_sum','total_reserv_mean', 'total_reserv_dt_diff_mean', 
                                   'date_int','var_max_lat', 'var_max_long', 'lon_plus_lat','visitors']]
data_val = val_data[['id','air_store_id', 'visit_date',
                                   'holiday_flg','dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear',
                                   'min_visitors', 'mean_visitors',
                                  'median_visitors', 'max_visitors', 'count_observations',
                                  'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
                                  'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 
                                  'total_reserv_sum','total_reserv_mean', 'total_reserv_dt_diff_mean', 
                                   'date_int','var_max_lat', 'var_max_long', 'lon_plus_lat','visitors']]
data_test = test_data[['id','air_store_id', 'visit_date',
                                   'holiday_flg','dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear',
                                   'min_visitors', 'mean_visitors',
                                  'median_visitors', 'max_visitors', 'count_observations',
                                  'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
                                  'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 
                                  'total_reserv_sum','total_reserv_mean', 'total_reserv_dt_diff_mean', 
                                   'date_int','var_max_lat', 'var_max_long', 'lon_plus_lat','visitors']]

In [6]:
data_train.head()

,id,air_store_id,visit_date,holiday_flg,dow,year,month,air_genre_name0,air_area_name0,air_genre_name1,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,visitors
0,air_ba937bf13d40fb24_2016-01-13,air_ba937bf13d40fb24,2016-01-13,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,25
1,air_ba937bf13d40fb24_2016-01-20,air_ba937bf13d40fb24,2016-01-20,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160120,8.362564,4.521799,175.409667,31
2,air_ba937bf13d40fb24_2016-01-27,air_ba937bf13d40fb24,2016-01-27,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160127,8.362564,4.521799,175.409667,24
3,air_ba937bf13d40fb24_2016-02-03,air_ba937bf13d40fb24,2016-02-03,0,2,2016,2,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160203,8.362564,4.521799,175.409667,18
4,air_ba937bf13d40fb24_2016-02-10,air_ba937bf13d40fb24,2016-02-10,0,2,2016,2,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160210,8.362564,4.521799,175.409667,32


In [7]:
full_data_train.is_copy=False
data_train.is_copy=False
data_val.is_copy=False
data_test.is_copy=False

In [8]:
full_data_train.air_store_id.replace(sorted(full_data_train.air_store_id.unique()),np.arange(0,len(set(full_data_train.air_store_id)),1),inplace=True)


In [9]:
full_data_train.head()

,id,air_store_id,visit_date,holiday_flg,dow,year,month,air_genre_name0,air_area_name0,air_genre_name1,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,visitors
0,air_ba937bf13d40fb24_2016-01-13,599,2016-01-13,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,25
1,air_ba937bf13d40fb24_2016-01-20,599,2016-01-20,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160120,8.362564,4.521799,175.409667,31
2,air_ba937bf13d40fb24_2016-01-27,599,2016-01-27,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160127,8.362564,4.521799,175.409667,24
3,air_ba937bf13d40fb24_2016-02-03,599,2016-02-03,0,2,2016,2,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160203,8.362564,4.521799,175.409667,18
4,air_ba937bf13d40fb24_2016-02-10,599,2016-02-10,0,2,2016,2,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160210,8.362564,4.521799,175.409667,32


In [10]:
data_train.air_store_id.replace(sorted(data_train.air_store_id.unique()),np.arange(0,len(set(data_train.air_store_id)),1),inplace=True)


In [11]:
data_train.head()

,id,air_store_id,visit_date,holiday_flg,dow,year,month,air_genre_name0,air_area_name0,air_genre_name1,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,visitors
0,air_ba937bf13d40fb24_2016-01-13,599,2016-01-13,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,25
1,air_ba937bf13d40fb24_2016-01-20,599,2016-01-20,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160120,8.362564,4.521799,175.409667,31
2,air_ba937bf13d40fb24_2016-01-27,599,2016-01-27,0,2,2016,1,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160127,8.362564,4.521799,175.409667,24
3,air_ba937bf13d40fb24_2016-02-03,599,2016-02-03,0,2,2016,2,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160203,8.362564,4.521799,175.409667,18
4,air_ba937bf13d40fb24_2016-02-10,599,2016-02-10,0,2,2016,2,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160210,8.362564,4.521799,175.409667,32


In [12]:
data_val.air_store_id.replace(sorted(data_train.air_store_id.unique()),np.arange(0,len(set(data_train.air_store_id)),1),inplace=True)


In [13]:
data_val.head()

,id,air_store_id,visit_date,holiday_flg,dow,year,month,air_genre_name0,air_area_name0,air_genre_name1,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,visitors
0,air_ba937bf13d40fb24_2017-03-29,air_ba937bf13d40fb24,2017-03-29,0,2,2017,3,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170329,8.362564,4.521799,175.409667,20
1,air_ba937bf13d40fb24_2017-04-05,air_ba937bf13d40fb24,2017-04-05,0,2,2017,4,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170405,8.362564,4.521799,175.409667,19
2,air_ba937bf13d40fb24_2017-04-12,air_ba937bf13d40fb24,2017-04-12,0,2,2017,4,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170412,8.362564,4.521799,175.409667,7
3,air_ba937bf13d40fb24_2017-04-19,air_ba937bf13d40fb24,2017-04-19,0,2,2017,4,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170419,8.362564,4.521799,175.409667,11
4,air_ba937bf13d40fb24_2017-03-30,air_ba937bf13d40fb24,2017-03-30,0,3,2017,3,4,7,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170330,8.362564,4.521799,175.409667,20


In [14]:
data_test.air_store_id.replace(sorted(data_train.air_store_id.unique()),np.arange(0,len(set(data_train.air_store_id)),1),inplace=True)

In [15]:
full_data_train.dow.replace(sorted(data_train.dow.unique()),np.arange(0,len(set(train_data.dow)),1),inplace=True)
data_train.dow.replace(sorted(data_train.dow.unique()),np.arange(0,len(set(train_data.dow)),1),inplace=True)
data_val.dow.replace(sorted(data_train.dow.unique()),np.arange(0,len(set(train_data.dow)),1),inplace=True)
data_test.dow.replace(sorted(data_train.dow.unique()),np.arange(0,len(set(train_data.dow)),1),inplace=True)

In [16]:
data_test.head()

,id,air_store_id,visit_date,holiday_flg,dow,year,month,air_genre_name0,air_area_name0,air_genre_name1,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,0,6,2017,4,6,7,2,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170423,8.326629,4.519803,175.447598,0
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0,0,2017,4,6,7,2,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170424,8.326629,4.519803,175.447598,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,0,1,2017,4,6,7,2,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170425,8.326629,4.519803,175.447598,0
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,0,2,2017,4,6,7,2,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170426,8.326629,4.519803,175.447598,0
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,0,3,2017,4,6,7,2,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170427,8.326629,4.519803,175.447598,0


In [17]:
full_data_train.year.replace(sorted(data_train.year.unique()),np.arange(0,len(set(data_train.year)),1),inplace=True)
data_train.year.replace(sorted(data_train.year.unique()),np.arange(0,len(set(data_train.year)),1),inplace=True)
data_val.year.replace(sorted(data_train.year.unique()),np.arange(0,len(set(data_train.year)),1),inplace=True)
data_test.year.replace(sorted(data_train.year.unique()),np.arange(0,len(set(data_train.year)),1),inplace=True)

In [89]:
full_data_train.month.replace(sorted(data_train.month.unique()),np.arange(0,len(set(data_train.month)),1),inplace=True)
data_train.month.replace(sorted(data_train.month.unique()),np.arange(0,len(set(data_train.month)),1),inplace=True)
data_val.month.replace(sorted(data_train.month.unique()),np.arange(0,len(set(data_train.month)),1),inplace=True)
data_test.month.replace(sorted(data_train.month.unique()),np.arange(0,len(set(data_train.month)),1),inplace=True)

In [90]:
full_data_train.air_genre_name0.replace(sorted(data_train.air_genre_name0.unique()),np.arange(0,len(set(data_train.air_genre_name0)),1),inplace=True)
data_train.air_genre_name0.replace(sorted(data_train.air_genre_name0.unique()),np.arange(0,len(set(data_train.air_genre_name0)),1),inplace=True)
data_val.air_genre_name0.replace(sorted(data_train.air_genre_name0.unique()),np.arange(0,len(set(data_train.air_genre_name0)),1),inplace=True)
data_test.air_genre_name0.replace(sorted(data_train.air_genre_name0.unique()),np.arange(0,len(set(data_train.air_genre_name0)),1),inplace=True)

In [91]:
full_data_train.air_genre_name1.replace(sorted(data_train.air_genre_name1.unique()),np.arange(0,len(set(data_train.air_genre_name1)),1),inplace=True)
data_train.air_genre_name1.replace(sorted(data_train.air_genre_name1.unique()),np.arange(0,len(set(data_train.air_genre_name1)),1),inplace=True)
data_val.air_genre_name1.replace(sorted(data_train.air_genre_name1.unique()),np.arange(0,len(set(data_train.air_genre_name1)),1),inplace=True)
data_test.air_genre_name1.replace(sorted(data_train.air_genre_name1.unique()),np.arange(0,len(set(data_train.air_genre_name1)),1),inplace=True)

In [92]:
full_data_train.air_area_name0.replace(sorted(data_train.air_area_name0.unique()),np.arange(0,len(set(data_train.air_area_name0)),1),inplace=True)
data_train.air_area_name0.replace(sorted(data_train.air_area_name0.unique()),np.arange(0,len(set(data_train.air_area_name0)),1),inplace=True)
data_val.air_area_name0.replace(sorted(data_train.air_area_name0.unique()),np.arange(0,len(set(data_train.air_area_name0)),1),inplace=True)
data_test.air_area_name0.replace(sorted(data_train.air_area_name0.unique()),np.arange(0,len(set(data_train.air_area_name0)),1),inplace=True)

In [93]:
full_data_train.air_genre_name2.replace(sorted(data_train.air_genre_name2.unique()),np.arange(0,len(set(data_train.air_genre_name2)),1),inplace=True)
data_train.air_genre_name2.replace(sorted(data_train.air_genre_name2.unique()),np.arange(0,len(set(data_train.air_genre_name2)),1),inplace=True)
data_val.air_genre_name2.replace(sorted(data_train.air_genre_name2.unique()),np.arange(0,len(set(data_train.air_genre_name2)),1),inplace=True)
data_test.air_genre_name2.replace(sorted(data_train.air_genre_name2.unique()),np.arange(0,len(set(data_train.air_genre_name2)),1),inplace=True)

In [94]:
full_data_train.air_area_name1.replace(sorted(data_train.air_area_name1.unique()),np.arange(0,len(set(data_train.air_area_name1)),1),inplace=True)
data_train.air_area_name1.replace(sorted(data_train.air_area_name1.unique()),np.arange(0,len(set(data_train.air_area_name1)),1),inplace=True)
data_val.air_area_name1.replace(sorted(data_train.air_area_name1.unique()),np.arange(0,len(set(data_train.air_area_name1)),1),inplace=True)
data_test.air_area_name1.replace(sorted(data_train.air_area_name1.unique()),np.arange(0,len(set(data_train.air_area_name1)),1),inplace=True)

In [95]:
full_data_train.air_area_name2.replace(sorted(data_train.air_area_name2.unique()),np.arange(0,len(set(data_train.air_area_name2)),1),inplace=True)
data_train.air_area_name2.replace(sorted(data_train.air_area_name2.unique()),np.arange(0,len(set(data_train.air_area_name2)),1),inplace=True)
data_val.air_area_name2.replace(sorted(data_train.air_area_name2.unique()),np.arange(0,len(set(data_train.air_area_name2)),1),inplace=True)
data_test.air_area_name2.replace(sorted(data_train.air_area_name2.unique()),np.arange(0,len(set(data_train.air_area_name2)),1),inplace=True)

In [96]:
full_data_train.air_area_name3.replace(sorted(data_train.air_area_name3.unique()),np.arange(0,len(set(data_train.air_area_name3)),1),inplace=True)
data_train.air_area_name3.replace(sorted(data_train.air_area_name3.unique()),np.arange(0,len(set(data_train.air_area_name3)),1),inplace=True)
data_val.air_area_name3.replace(sorted(data_train.air_area_name3.unique()),np.arange(0,len(set(data_train.air_area_name3)),1),inplace=True)
data_test.air_area_name3.replace(sorted(data_train.air_area_name3.unique()),np.arange(0,len(set(data_train.air_area_name3)),1),inplace=True)

In [97]:
full_data_train.air_area_name4.replace(sorted(data_train.air_area_name4.unique()),np.arange(0,len(set(data_train.air_area_name4)),1),inplace=True)
data_train.air_area_name4.replace(sorted(data_train.air_area_name4.unique()),np.arange(0,len(set(data_train.air_area_name4)),1),inplace=True)
data_val.air_area_name4.replace(sorted(data_train.air_area_name4.unique()),np.arange(0,len(set(data_train.air_area_name4)),1),inplace=True)
data_test.air_area_name4.replace(sorted(data_train.air_area_name4.unique()),np.arange(0,len(set(data_train.air_area_name4)),1),inplace=True)

In [98]:
full_data_train.air_area_name5.replace(sorted(data_train.air_area_name5.unique()),np.arange(0,len(set(data_train.air_area_name5)),1),inplace=True)
data_train.air_area_name5.replace(sorted(data_train.air_area_name5.unique()),np.arange(0,len(set(data_train.air_area_name5)),1),inplace=True)
data_val.air_area_name5.replace(sorted(data_train.air_area_name5.unique()),np.arange(0,len(set(data_train.air_area_name5)),1),inplace=True)
data_test.air_area_name5.replace(sorted(data_train.air_area_name5.unique()),np.arange(0,len(set(data_train.air_area_name5)),1),inplace=True)



In [99]:
full_data_train.air_area_name6.replace(sorted(data_train.air_area_name6.unique()),np.arange(0,len(set(data_train.air_area_name6)),1),inplace=True)
data_train.air_area_name6.replace(sorted(data_train.air_area_name6.unique()),np.arange(0,len(set(data_train.air_area_name6)),1),inplace=True)
data_val.air_area_name6.replace(sorted(data_train.air_area_name6.unique()),np.arange(0,len(set(data_train.air_area_name6)),1),inplace=True)
data_test.air_area_name6.replace(sorted(data_train.air_area_name6.unique()),np.arange(0,len(set(data_train.air_area_name6)),1),inplace=True)



In [100]:
full_data_train.dayofquarter.replace(sorted(data_train.dayofquarter.unique()),np.arange(0,len(set(data_train.dayofquarter)),1),inplace=True)
data_train.dayofquarter.replace(sorted(data_train.dayofquarter.unique()),np.arange(0,len(set(data_train.dayofquarter)),1),inplace=True)
data_val.dayofquarter.replace(sorted(data_train.dayofquarter.unique()),np.arange(0,len(set(data_train.dayofquarter)),1),inplace=True)
data_test.dayofquarter.replace(sorted(data_train.dayofquarter.unique()),np.arange(0,len(set(data_train.dayofquarter)),1),inplace=True)



In [101]:
full_data_train.quarter.replace(sorted(data_train.quarter.unique()),np.arange(0,len(set(data_train.quarter)),1),inplace=True)
data_train.quarter.replace(sorted(data_train.quarter.unique()),np.arange(0,len(set(data_train.quarter)),1),inplace=True)
data_val.quarter.replace(sorted(data_train.quarter.unique()),np.arange(0,len(set(data_train.quarter)),1),inplace=True)
data_test.quarter.replace(sorted(data_train.quarter.unique()),np.arange(0,len(set(data_train.quarter)),1),inplace=True)



In [102]:
full_data_train.day.replace(sorted(data_train.day.unique()),np.arange(0,len(set(data_train.day)),1),inplace=True)
data_train.day.replace(sorted(data_train.day.unique()),np.arange(0,len(set(data_train.day)),1),inplace=True)
data_val.day.replace(sorted(data_train.day.unique()),np.arange(0,len(set(data_train.day)),1),inplace=True)
data_test.day.replace(sorted(data_train.day.unique()),np.arange(0,len(set(data_train.day)),1),inplace=True)



In [103]:
full_data_train.dayofyear.replace(sorted(data_train.dayofyear.unique()),np.arange(0,len(set(data_train.dayofyear)),1),inplace=True)
data_train.dayofyear.replace(sorted(data_train.dayofyear.unique()),np.arange(0,len(set(data_train.dayofyear)),1),inplace=True)
data_val.dayofyear.replace(sorted(data_train.dayofyear.unique()),np.arange(0,len(set(data_train.dayofyear)),1),inplace=True)
data_test.dayofyear.replace(sorted(data_train.dayofyear.unique()),np.arange(0,len(set(data_train.dayofyear)),1),inplace=True)

In [104]:
full_data_train.to_csv('./data/full_data_train.csv',index=False)
data_train.to_csv('./data/data_train.csv',index=False)
data_val.to_csv('./data/data_val.csv',index=False)
data_test.to_csv('./data/data_test.csv',index=False)

In [59]:
type(data_train['id'].values)

numpy.ndarray

In [8]:
len(train_data.columns),len(test_data.columns)

(61, 59)

In [19]:
store_set = set(train_data.air_store_id)
dow_set = set(train_data.day_of_week)
year_set = set(train_data.year)
month_set = set(train_data.month)
doq_set = set(train_data.dayofquarter)
quarter_set = set(train_data.quarter)
day_set = set(train_data.day)
doy_set = set(train_data.dayofyear)
gen0_set = set(train_data.air_genre_name0)
gen1_set = set(train_data.air_genre_name1)
gen2_set = set(train_data.air_genre_name2)
area0_set = set(train_data.air_area_name0)
area1_set = set(train_data.air_area_name1)
area2_set = set(train_data.air_area_name2)
area3_set = set(train_data.air_area_name3)
area4_set = set(train_data.air_area_name4)
area5_set = set(train_data.air_area_name5)
area6_set = set(train_data.air_area_name6)

In [20]:
store_index_dict = {store:i for i,store in enumerate(store_set)}
dow_index_dict = {dow:i for i,dow in enumerate(dow_set)}
year_index_dict = {year:i for i,year in enumerate(year_set)}
month_index_dict = {month:i for i,month in enumerate(month_set)}
doq_index_dict = {doq:i for i,doq in enumerate(doq_set)}
quarter_index_dict = {quarter:i for i,quarter in enumerate(quarter_set)}
day_index_dict = {day:i for i,day in enumerate(day_set)}
doy_index_dict = {doy:i for i,doy in enumerate(doy_set)}
gen0_index_dict = {gen0:i for i,gen0 in enumerate(gen0_set)}
gen1_index_dict = {gen1:i for i,gen1 in enumerate(gen1_set)}
gen2_index_dict = {gen2:i for i,gen2 in enumerate(gen2_set)}
area0_index_dict = {area0:i for i,area0 in enumerate(area0_set)}
area1_index_dict = {area1:i for i,area1 in enumerate(area1_set)}
area2_index_dict = {area2:i for i,area2 in enumerate(area2_set)}
area3_index_dict = {area3:i for i,area3 in enumerate(area3_set)}
area4_index_dict = {area4:i for i,area4 in enumerate(area4_set)}
area5_index_dict = {area5:i for i,area5 in enumerate(area5_set)}
area6_index_dict = {area6:i for i,area6 in enumerate(area6_set)}

In [21]:
def store_index(data):
    return store_index_dict[data]
def dow_index(data):
    return dow_index_dict[data]
def year_index(data):
    return year_index_dict[data]
def month_index(data):
    return month_index_dict[data]
def doq_index(data):
    return doq_index_dict[data]
def quarter_index(data):
    return quarter_index_dict[data]
def day_index(data):
    return day_index_dict[data]
def doy_index(data):
    return doy_index_dict[data]
def gen0_index(data):
    return gen0_index_dict[data]
def gen1_index(data):
    return gen1_index_dict[data]
def gen2_index(data):
    return gen2_index_dict[data]
def area0_index(data):
    return area0_index_dict[data]
def area1_index(data):
    return area1_index_dict[data]
def area2_index(data):
    return area2_index_dict[data]
def area3_index(data):
    return area3_index_dict[data]
def area4_index(data):
    return area4_index_dict[data]
def area5_index(data):
    return area5_index_dict[data]
def area6_index(data):
    return area6_index_dict[data]

In [22]:
full_data_train.columns

Index(['id', 'air_store_id', 'visit_date', 'holiday_flg', 'dow', 'year',
       'month', 'air_genre_name0', 'air_area_name0', 'air_genre_name1',
       'air_area_name1', 'air_genre_name2', 'air_area_name2', 'air_area_name3',
       'air_area_name4', 'air_area_name5', 'air_area_name6', 'dayofquarter',
       'quarter', 'day', 'dayofyear', 'min_visitors', 'mean_visitors',
       'median_visitors', 'max_visitors', 'count_observations', 'rs1_x',
       'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y',
       'total_reserv_sum', 'total_reserv_mean', 'total_reserv_dt_diff_mean',
       'date_int', 'var_max_lat', 'var_max_long', 'lon_plus_lat', 'visitors'],
      dtype='object')

In [23]:
full_train_df = full_data_train.drop(['dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear'],axis=1)
train_df = data_train.drop(['dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear'],axis=1)
val_df = data_val.drop(['dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear'],axis=1)
test_df = data_test.drop(['dow', 'year', 'month','air_genre_name0', 
                                   'air_area_name0', 'air_genre_name1',
                                   'air_area_name1', 'air_genre_name2', 'air_area_name2',
                                   'air_area_name3','air_area_name4','air_area_name5',
                                   'air_area_name6','dayofquarter', 'quarter', 'day', 'dayofyear'],axis=1)

In [24]:
full_train_df.head()

,id,air_store_id,visit_date,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,rs1_x,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,visitors
0,air_ba937bf13d40fb24_2016-01-13,air_ba937bf13d40fb24,2016-01-13,0,7.0,23.84375,25.0,57.0,64.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,25
1,air_ba937bf13d40fb24_2016-01-20,air_ba937bf13d40fb24,2016-01-20,0,7.0,23.84375,25.0,57.0,64.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160120,8.362564,4.521799,175.409667,31
2,air_ba937bf13d40fb24_2016-01-27,air_ba937bf13d40fb24,2016-01-27,0,7.0,23.84375,25.0,57.0,64.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160127,8.362564,4.521799,175.409667,24
3,air_ba937bf13d40fb24_2016-02-03,air_ba937bf13d40fb24,2016-02-03,0,7.0,23.84375,25.0,57.0,64.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160203,8.362564,4.521799,175.409667,18
4,air_ba937bf13d40fb24_2016-02-10,air_ba937bf13d40fb24,2016-02-10,0,7.0,23.84375,25.0,57.0,64.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20160210,8.362564,4.521799,175.409667,32


In [25]:
full_data_train.columns

Index(['id', 'air_store_id', 'visit_date', 'holiday_flg', 'dow', 'year',
       'month', 'air_genre_name0', 'air_area_name0', 'air_genre_name1',
       'air_area_name1', 'air_genre_name2', 'air_area_name2', 'air_area_name3',
       'air_area_name4', 'air_area_name5', 'air_area_name6', 'dayofquarter',
       'quarter', 'day', 'dayofyear', 'min_visitors', 'mean_visitors',
       'median_visitors', 'max_visitors', 'count_observations', 'rs1_x',
       'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y',
       'total_reserv_sum', 'total_reserv_mean', 'total_reserv_dt_diff_mean',
       'date_int', 'var_max_lat', 'var_max_long', 'lon_plus_lat', 'visitors'],
      dtype='object')

In [26]:
full_train_df['store'] = full_data_train.air_store_id.apply(store_index)
full_train_df['dow'] = full_data_train.dow.apply(dow_index)
full_train_df['year'] = full_data_train.year.apply(year_index)
full_train_df['month'] = full_data_train.month.apply(month_index)
full_train_df['doq'] = full_data_train.dayofquarter.apply(doq_index)
full_train_df['quarter'] = full_data_train.quarter.apply(quarter_index)
full_train_df['day'] = full_data_train.day.apply(day_index)
full_train_df['doy'] = full_data_train.dayofyear.apply(doy_index)
full_train_df['gen0'] = full_data_train.air_genre_name0.apply(gen0_index)
full_train_df['gen1'] = full_data_train.air_genre_name1.apply(gen1_index)
full_train_df['gen2'] = full_data_train.air_genre_name2.apply(gen2_index)
full_train_df['area0'] = full_data_train.air_area_name0.apply(area0_index)
full_train_df['area1'] = full_data_train.air_area_name1.apply(area1_index)
full_train_df['area2'] = full_data_train.air_area_name2.apply(area2_index)
full_train_df['area3'] = full_data_train.air_area_name3.apply(area3_index)
full_train_df['area4'] = full_data_train.air_area_name4.apply(area4_index)
full_train_df['area5'] = full_data_train.air_area_name5.apply(area5_index)
full_train_df['area6'] = full_data_train.air_area_name6.apply(area6_index)

In [27]:
train_df['store'] = data_train.air_store_id.apply(store_index)
train_df['dow'] = data_train.dow.apply(dow_index)
train_df['year'] = data_train.year.apply(year_index)
train_df['month'] = data_train.month.apply(month_index)
train_df['doq'] = data_train.dayofquarter.apply(doq_index)
train_df['quarter'] = data_train.quarter.apply(quarter_index)
train_df['day'] = data_train.day.apply(day_index)
train_df['doy'] = data_train.dayofyear.apply(doy_index)
train_df['gen0'] = data_train.air_genre_name0.apply(gen0_index)
train_df['gen1'] = data_train.air_genre_name1.apply(gen1_index)
train_df['gen2'] = data_train.air_genre_name2.apply(gen2_index)
train_df['area0'] = data_train.air_area_name0.apply(area0_index)
train_df['area1'] = data_train.air_area_name1.apply(area1_index)
train_df['area2'] = data_train.air_area_name2.apply(area2_index)
train_df['area3'] = data_train.air_area_name3.apply(area3_index)
train_df['area4'] = data_train.air_area_name4.apply(area4_index)
train_df['area5'] = data_train.air_area_name5.apply(area5_index)
train_df['area6'] = data_train.air_area_name6.apply(area6_index)

In [28]:
val_df['store'] = data_val.air_store_id.apply(store_index)
val_df['dow'] = data_val.dow.apply(dow_index)
val_df['year'] = data_val.year.apply(year_index)
val_df['month'] = data_val.month.apply(month_index)
val_df['doq'] = data_val.dayofquarter.apply(doq_index)
val_df['quarter'] = data_val.quarter.apply(quarter_index)
val_df['day'] = data_val.day.apply(day_index)
val_df['doy'] = data_val.dayofyear.apply(doy_index)
val_df['gen0'] = data_val.air_genre_name0.apply(gen0_index)
val_df['gen1'] = data_val.air_genre_name1.apply(gen1_index)
val_df['gen2'] = data_val.air_genre_name2.apply(gen2_index)
val_df['area0'] = data_val.air_area_name0.apply(area0_index)
val_df['area1'] = data_val.air_area_name1.apply(area1_index)
val_df['area2'] = data_val.air_area_name2.apply(area2_index)
val_df['area3'] = data_val.air_area_name3.apply(area3_index)
val_df['area4'] = data_val.air_area_name4.apply(area4_index)
val_df['area5'] = data_val.air_area_name5.apply(area5_index)
val_df['area6'] = data_val.air_area_name6.apply(area6_index)

In [29]:
test_df['store'] = data_test.air_store_id.apply(store_index)
test_df['dow'] = data_test.dow.apply(dow_index)
test_df['year'] = data_test.year.apply(year_index)
test_df['month'] = data_test.month.apply(month_index)
test_df['doq'] = data_test.dayofquarter.apply(doq_index)
test_df['quarter'] = data_test.quarter.apply(quarter_index)
test_df['day'] = data_test.day.apply(day_index)
test_df['doy'] = data_test.dayofyear.apply(doy_index)
test_df['gen0'] = data_test.air_genre_name0.apply(gen0_index)
test_df['gen1'] = data_test.air_genre_name1.apply(gen1_index)
test_df['gen2'] = data_test.air_genre_name2.apply(gen2_index)
test_df['area0'] = data_test.air_area_name0.apply(area0_index)
test_df['area1'] = data_test.air_area_name1.apply(area1_index)
test_df['area2'] = data_test.air_area_name2.apply(area2_index)
test_df['area3'] = data_test.air_area_name3.apply(area3_index)
test_df['area4'] = data_test.air_area_name4.apply(area4_index)
test_df['area5'] = data_test.air_area_name5.apply(area5_index)
test_df['area6'] = data_test.air_area_name6.apply(area6_index)

In [131]:
full_train_df.to_csv('./data/full_train_df.csv',index=False)
train_df.to_csv('./data/train_df.csv',index=False)
val_df.to_csv('./data/val_df.csv',index=False)
test_df.to_csv('./data/test_df.csv',index=False)

In [115]:
def get_month(data):
    return data-1
def get_day(data):
    return data-1
def get_doq(data):
    return data-1
def get_doy(data):
    return data-1
def get_year(data):
    if data == 2016:
        return 0
    else:
        return 1

In [18]:
train_data.month.iloc[:3].apply(get_month)

0    0
1    0
2    0
Name: month, dtype: int64

In [19]:
train_data['month2'] = train_data.month.apply(get_month)
val_data['month2'] = val_data.month.apply(get_month)
test_data['month2'] = test_data.month.apply(get_month)
full_train_data['month2'] = full_train_data.month.apply(get_month)

In [80]:
train_data['day'] = train_data.day.apply(get_day)
val_data['day'] = val_data.day.apply(get_day)
test_data['day'] = test_data.day.apply(get_day)
full_train_data['day'] = full_train_data.day.apply(get_day)

In [98]:
train_data['dayofquarter'] = train_data.dayofquarter.apply(get_doq)
val_data['dayofquarter'] = val_data.dayofquarter.apply(get_doq)
test_data['dayofquarter'] = test_data.dayofquarter.apply(get_doq)
full_train_data['dayofquarter'] = full_train_data.dayofquarter.apply(get_doq)

train_data['dayofyear'] = train_data.dayofyear.apply(get_doy)
val_data['dayofyear'] = val_data.dayofyear.apply(get_doy)
test_data['dayofyear'] = test_data.dayofyear.apply(get_doy)
full_train_data['dayofyear'] = full_train_data.dayofyear.apply(get_doy)

In [116]:
train_data['year'] = train_data.year.apply(get_year)
val_data['year'] = val_data.year.apply(get_year)
test_data['year'] = test_data.year.apply(get_year)
full_train_data['year'] = full_train_data.year.apply(get_year)



In [117]:
train_data['quarter'] = train_data.quarter.apply(get_day)
val_data['quarter'] = val_data.quarter.apply(get_day)
test_data['quarter'] = test_data.quarter.apply(get_day)
full_train_data['quarter'] = full_train_data.quarter.apply(get_day)


In [131]:
set(train_data.air_store_id2)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [36]:
ytrain = train_data.visitors.apply(np.log1p)
yval = train_data.visitors.apply(np.log1p)
fytrain = full_train_data.visitors.apply(np.log1p)

In [26]:
train_data.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'year', 'month',
       'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors',
       'median_visitors', 'max_visitors', 'count_observations',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'air_genre_name0', 'air_area_name0', 'air_genre_name1',
       'air_area_name1', 'air_genre_name2', 'air_area_name2',
       'air_genre_name3', 'air_area_name3', 'air_genre_name4',
       'air_area_name4', 'air_genre_name5', 'air_area_name5',
       'air_genre_name6', 'air_area_name6', 'air_genre_name7',
       'air_area_name7', 'air_genre_name8', 'air_area_name8',
       'air_genre_name9', 'air_area_name9', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
       'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int',
       'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2', 'date',
       'dayofquarter', 'quarter', 'day', 'dayofyear', 

In [30]:
xtrain_dict = {
    'air_store_id2':train_data.air_store_id2.values,
    'dow':train_data.dow.values,
    'gen0':train_data.air_genre_name0.values,
    'gen1':train_data.air_genre_name1.values,
    'gen2':train_data.air_genre_name2.values,
    'area0':train_data.air_area_name0.values,
    'area1':train_data.air_area_name1.values,
    'area2':train_data.air_area_name2.values,
    'area3':train_data.air_area_name3.values,
    'area4':train_data.air_area_name4.values,
    'area5':train_data.air_area_name5.values,
    'area6':train_data.air_area_name6.values,
    'month2':train_data.month2.values,
    'year':train_data.year.values,
    'quarter':train_data.quarter.values,
    'doy':train_data.dayofyear.values,
    'doq':train_data.dayofquarter.values,
    'day':train_data.day.values,
    'holi':train_data.holiday_flg.values
}
full_xtrain_dict = {
    'air_store_id2':full_train_data.air_store_id2.values,
    'dow':full_train_data.dow.values,
    'gen0':full_train_data.air_genre_name0.values,
    'gen1':full_train_data.air_genre_name1.values,
    'gen2':full_train_data.air_genre_name2.values,
    'area0':full_train_data.air_area_name0.values,
    'area1':full_train_data.air_area_name1.values,
    'area2':full_train_data.air_area_name2.values,
    'area3':full_train_data.air_area_name3.values,
    'area4':full_train_data.air_area_name4.values,
    'area5':full_train_data.air_area_name5.values,
    'area6':full_train_data.air_area_name6.values,
    'month2':full_train_data.month2.values,
    'year':full_train_data.year.values,
    'quarter':full_train_data.quarter.values,
    'doy':full_train_data.dayofyear.values,
    'doq':full_train_data.dayofquarter.values,
    'day':full_train_data.day.values,
    'holi':full_train_data.holiday_flg.values
}
xval_dict = {
    'air_store_id2':val_data.air_store_id2.values,
    'dow':val_data.dow.values,
    'gen0':val_data.air_genre_name0.values,
    'gen1':val_data.air_genre_name1.values,
    'gen2':val_data.air_genre_name2.values,
    'area0':val_data.air_area_name0.values,
    'area1':val_data.air_area_name1.values,
    'area2':val_data.air_area_name2.values,
    'area3':val_data.air_area_name3.values,
    'area4':val_data.air_area_name4.values,
    'area5':val_data.air_area_name5.values,
    'area6':val_data.air_area_name6.values,
    'month2':val_data.month2.values,
    'year':val_data.year.values,
    'quarter':val_data.quarter.values,
    'doy':val_data.dayofyear.values,
    'doq':val_data.dayofquarter.values,
    'day':val_data.day.values,
    'holi':val_data.holiday_flg.values
}
xval_dict_for_val = {
    'air_store_id2':Variable(torch.LongTensor(val_data.air_store_id2.values),requires_grad=False),
    'dow':Variable(torch.LongTensor(val_data.dow.values),requires_grad=False),
    'gen0':Variable(torch.LongTensor(val_data.air_genre_name0.values),requires_grad=False),
    'gen1':Variable(torch.LongTensor(val_data.air_genre_name1.values),requires_grad=False),
    'gen2':Variable(torch.LongTensor(val_data.air_genre_name2.values),requires_grad=False),
    'area0':Variable(torch.LongTensor(val_data.air_area_name0.values),requires_grad=False),
    'area1':Variable(torch.LongTensor(val_data.air_area_name1.values),requires_grad=False),
    'area2':Variable(torch.LongTensor(val_data.air_area_name2.values),requires_grad=False),
    'area3':Variable(torch.LongTensor(val_data.air_area_name3.values),requires_grad=False),
    'area4':Variable(torch.LongTensor(val_data.air_area_name4.values),requires_grad=False),
    'area5':Variable(torch.LongTensor(val_data.air_area_name5.values),requires_grad=False),
    'area6':Variable(torch.LongTensor(val_data.air_area_name6.values),requires_grad=False),
    'month2':Variable(torch.LongTensor(val_data.month2.values),requires_grad=False),
    'year':Variable(torch.LongTensor(val_data.year.values),requires_grad=False),
    'quarter':Variable(torch.LongTensor(val_data.quarter.values),requires_grad=False),
    'doy':Variable(torch.LongTensor(val_data.dayofyear.values),requires_grad=False),
    'doq':Variable(torch.LongTensor(val_data.dayofquarter.values),requires_grad=False),
    'day':Variable(torch.LongTensor(val_data.day.values),requires_grad=False),
    'holi':Variable(torch.LongTensor(val_data.holiday_flg.values),requires_grad=False)
}
xtest_dict = {
    'air_store_id2':Variable(torch.LongTensor(test_data.air_store_id2.values),requires_grad=False),
    'dow':Variable(torch.LongTensor(test_data.dow.values),requires_grad=False),
    'gen0':Variable(torch.LongTensor(test_data.air_genre_name0.values),requires_grad=False),
    'gen1':Variable(torch.LongTensor(test_data.air_genre_name1.values),requires_grad=False),
    'gen2':Variable(torch.LongTensor(test_data.air_genre_name2.values),requires_grad=False),
    'area0':Variable(torch.LongTensor(test_data.air_area_name0.values),requires_grad=False),
    'area1':Variable(torch.LongTensor(test_data.air_area_name1.values),requires_grad=False),
    'area2':Variable(torch.LongTensor(test_data.air_area_name2.values),requires_grad=False),
    'area3':Variable(torch.LongTensor(test_data.air_area_name3.values),requires_grad=False),
    'area4':Variable(torch.LongTensor(test_data.air_area_name4.values),requires_grad=False),
    'area5':Variable(torch.LongTensor(test_data.air_area_name5.values),requires_grad=False),
    'area6':Variable(torch.LongTensor(test_data.air_area_name6.values),requires_grad=False),
    'month2':Variable(torch.LongTensor(test_data.month2.values),requires_grad=False),
    'year':Variable(torch.LongTensor(test_data.year.values),requires_grad=False),
    'quarter':Variable(torch.LongTensor(test_data.quarter.values),requires_grad=False),
    'doy':Variable(torch.LongTensor(test_data.dayofyear.values),requires_grad=False),
    'doq':Variable(torch.LongTensor(test_data.dayofquarter.values),requires_grad=False),
    'day':Variable(torch.LongTensor(test_data.day.values),requires_grad=False),
    'holi':Variable(torch.LongTensor(test_data.holiday_flg.values),requires_grad=False)
}

AttributeError: 'DataFrame' object has no attribute 'month2'

In [31]:
xval_df = pd.concat([pd.DataFrame(xval_dict),yval.reset_index(drop=True)],axis=1)
xtrain_df = pd.DataFrame(xtrain_dict)
full_xtrain_df = pd.DataFrame(full_xtrain_dict)

NameError: name 'xval_dict' is not defined

In [120]:
def batch_up(batch_size,features,labels):
    assert len(features) == len(labels)
    output_batch = []
    
    sample_size = len(features)
    for start_i in range(0,sample_size,batch_size):
        end_i = start_i + batch_size
        batch_feature = features[start_i:end_i]
        batch_label = labels[start_i:end_i]
        output_batch.append((batch_feature,batch_label))
        
    return output_batch

batched_train_data = batch_up(256,xtrain_df,ytrain.values)
batched_full_train_data = batch_up(256,full_xtrain_df,fytrain.values)

In [121]:
def get_valid_data():
    data = xval_df.sample(100)
    y = data.visitors.values
    x = {
    'air_store_id2':Variable(torch.LongTensor(data.air_store_id2.values),requires_grad=False),
    'dow':Variable(torch.LongTensor(data.dow.values),requires_grad=False),
    'gen0':Variable(torch.LongTensor(data.gen0.values),requires_grad=False),
    'gen1':Variable(torch.LongTensor(data.gen1.values),requires_grad=False),
    'gen2':Variable(torch.LongTensor(data.gen2.values),requires_grad=False),
    'area0':Variable(torch.LongTensor(data.area0.values),requires_grad=False),
    'area1':Variable(torch.LongTensor(data.area1.values),requires_grad=False),
    'area2':Variable(torch.LongTensor(data.area2.values),requires_grad=False),
    'area3':Variable(torch.LongTensor(data.area3.values),requires_grad=False),
    'area4':Variable(torch.LongTensor(data.area4.values),requires_grad=False),
    'area5':Variable(torch.LongTensor(data.area5.values),requires_grad=False),
    'area6':Variable(torch.LongTensor(data.area6.values),requires_grad=False),
    'month2':Variable(torch.LongTensor(data.month2.values),requires_grad=False),
    'year':Variable(torch.LongTensor(data.year.values),requires_grad=False),
    'quarter':Variable(torch.LongTensor(data.quarter.values),requires_grad=False),
    'doy':Variable(torch.LongTensor(data.doy.values),requires_grad=False),
    'doq':Variable(torch.LongTensor(data.doq.values),requires_grad=False),
    'day':Variable(torch.LongTensor(data.day.values),requires_grad=False),
    'holi':Variable(torch.LongTensor(data.holi.values),requires_grad=False)
    }
    return x,y

In [122]:
def get_xv_dict(data):
    diction = {
    'air_store_id2':Variable(torch.LongTensor(data.air_store_id2.values)),
    'dow':Variable(torch.LongTensor(data.dow.values)),
    'gen0':Variable(torch.LongTensor(data.gen0.values)),
    'gen1':Variable(torch.LongTensor(data.gen1.values)),
    'gen2':Variable(torch.LongTensor(data.gen2.values)),
    'area0':Variable(torch.LongTensor(data.area0.values)),
    'area1':Variable(torch.LongTensor(data.area1.values)),
    'area2':Variable(torch.LongTensor(data.area2.values)),
    'area3':Variable(torch.LongTensor(data.area3.values)),
    'area4':Variable(torch.LongTensor(data.area4.values)),
    'area5':Variable(torch.LongTensor(data.area5.values)),
    'area6':Variable(torch.LongTensor(data.area6.values)),
    'month2':Variable(torch.LongTensor(data.month2.values)),
    'year':Variable(torch.LongTensor(data.year.values)),
    'quarter':Variable(torch.LongTensor(data.quarter.values)),
    'doy':Variable(torch.LongTensor(data.doy.values)),
    'doq':Variable(torch.LongTensor(data.doq.values)),
    'day':Variable(torch.LongTensor(data.day.values)),
    'holi':Variable(torch.LongTensor(data.holi.values))
    }
    return diction

In [123]:
len(batched_train_data),len(batched_full_train_data)

(895, 979)

In [124]:
def retrive_y(data):
    return np.exp(data)-1
def rmsle(preds,real):
    return np.sqrt(mean_squared_error(real,preds))
def valid_full(model):
    ypreds = model(xval_dict_for_val).detach().numpy()
    return rmsle(ypreds,yval.values)
def write_result(model,name='sub1.csv'):
    ypreds = model(xtest_dict).detach().numpy()
    ypreds = pd.Series(ypreds.reshape((-1,))).apply(retrive_y).values
    test_data['visitor'] = ypreds
    test_data[['id','visitor']].rename(columns={'visitor':'visitors'}).to_csv(name,index=False)

## define nn

In [125]:
set(xtrain_df.quarter)

{0, 1, 2, 3}

In [126]:
xtrain_df.columns

Index(['air_store_id2', 'area0', 'area1', 'area2', 'area3', 'area4', 'area5',
       'area6', 'day', 'doq', 'dow', 'doy', 'gen0', 'gen1', 'gen2', 'holi',
       'month2', 'quarter', 'year'],
      dtype='object')

In [127]:
print(len(set(xtrain_df.air_store_id2)),len(set(xtrain_df.area0)),len(set(xtrain_df.area1)),len(set(xtrain_df.area2)),
      len(set(xtrain_df.area3)),len(set(xtrain_df.area4)),len(set(xtrain_df.area5)),len(set(xtrain_df.area6)),
      len(set(xtrain_df.day)),len(set(xtrain_df.doq)),len(set(xtrain_df.dow)),len(set(xtrain_df.doy)),len(set(xtrain_df.gen0)),
      len(set(xtrain_df.gen1)),len(set(xtrain_df.gen2)),len(set(xtrain_df.holi)),
      len(set(xtrain_df.month2)),len(set(xtrain_df.quarter)),len(set(xtrain_df.year)))

821 9 7 53 8 101 10 2 31 92 7 366 14 10 3 2 12 4 2


In [128]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor,self).__init__()
        
        self.store_embeddor = nn.Embedding(821,15)
        self.gen0_embeddor = nn.Embedding(14,6)
        self.gen1_embeddor = nn.Embedding(10,5)
        self.gen2_embeddor = nn.Embedding(3,2)
        self.area0_embeddor = nn.Embedding(9,4)
        self.area1_embeddor = nn.Embedding(7,3)
        self.area2_embeddor = nn.Embedding(53,8)
        self.area3_embeddor = nn.Embedding(8,3)
        self.area4_embeddor = nn.Embedding(101,10)
        self.area5_embeddor = nn.Embedding(10,4)
        self.area6_embeddor = nn.Embedding(2,2)
        self.day_embeddor = nn.Embedding(31,8)
        self.doq_embeddor = nn.Embedding(92,10)
        self.dow_embeddor = nn.Embedding(7,3)
        self.doy_embeddor = nn.Embedding(366,15)
        self.holi_embeddor = nn.Embedding(2,2)
        self.month2_embeddor = nn.Embedding(12,5)
        self.quarter_embeddor = nn.Embedding(4,2)
        self.year_embeddor = nn.Embedding(2,2)
        
        self.e2h = nn.Linear(109,2000)
        self.bn0 = nn.BatchNorm1d(2000)
        self.h2h1 = nn.Linear(2000,1600)
        self.bn1 = nn.BatchNorm1d(1600)
        self.h2h2 = nn.Linear(1600,1300)
        self.bn2 = nn.BatchNorm1d(1300)
        self.h2h3 = nn.Linear(1300,1000)
        self.bn3 = nn.BatchNorm1d(1000)
        self.h2h4 = nn.Linear(1000,680)
        self.bn4 = nn.BatchNorm1d(680)
        self.h2h5 = nn.Linear(680,380)
        self.bn5 = nn.BatchNorm1d(380)
        self.h2h6 = nn.Linear(380,200)
        self.bn6 = nn.BatchNorm1d(200)
        self.h2h7 = nn.Linear(200,80)
        self.bn7 = nn.BatchNorm1d(80)
        self.h2o = nn.Linear(80,1)
        
        self.dropout = nn.Dropout(0.1)
        
        for layer in [self.e2h,self.h2h1,self.h2h2,self.h2h3,self.h2h4,self.h2h5,self.h2h6,self.h2h7,self.h2o]:
            self.init_wb(layer)
            
    def init_wb(self,layer):
        init.normal(layer.weight,mean=0.,std=0.1)
        init.constant(layer.bias,0)
        
    def forward(self,inputs):
        store_embeds = self.store_embeddor(inputs['air_store_id2']).view(-1,15)
        gen0_embeds = self.gen0_embeddor(inputs['gen0']).view(-1,6)
        gen1_embeds = self.gen1_embeddor(inputs['gen1']).view(-1,5)
        gen2_embeds = self.gen2_embeddor(inputs['gen2']).view(-1,2)
        area0_embeds = self.area0_embeddor(inputs['area0']).view(-1,4)
        area1_embeds = self.area1_embeddor(inputs['area1']).view(-1,3)
        area2_embeds = self.area2_embeddor(inputs['area2']).view(-1,8)
        area3_embeds = self.area3_embeddor(inputs['area3']).view(-1,3)
        area4_embeds = self.area4_embeddor(inputs['area4']).view(-1,10)
        area5_embeds = self.area5_embeddor(inputs['area5']).view(-1,4)
        area6_embeds = self.area6_embeddor(inputs['area6']).view(-1,2)
        day_embeds = self.day_embeddor(inputs['day']).view(-1,8)
        doq_embeds = self.doq_embeddor(inputs['doq']).view(-1,10)
        dow_embeds = self.dow_embeddor(inputs['dow']).view(-1,3)
        doy_embeds = self.doy_embeddor(inputs['doy']).view(-1,15)
        holi_embeds = self.holi_embeddor(inputs['holi']).view(-1,2)
        month2_embeds = self.month2_embeddor(inputs['month2']).view(-1,5)
        quarter_embeds = self.quarter_embeddor(inputs['quarter']).view(-1,2)
        year_embeds = self.year_embeddor(inputs['year']).view(-1,2)
        
        merged = torch.cat([store_embeds,gen0_embeds,gen1_embeds,gen2_embeds,area0_embeds,
                            area1_embeds,area2_embeds,area3_embeds,area4_embeds,area5_embeds,
                            area6_embeds,day_embeds,doq_embeds,dow_embeds,doy_embeds,holi_embeds,
                            month2_embeds,quarter_embeds,year_embeds],1)
        goods = self.dropout(self.bn0(self.e2h(merged)).clamp(min=0))
        goods = self.dropout(self.bn1(self.h2h1(goods)).clamp(min=0))
        goods = self.dropout(self.bn2(self.h2h2(goods)).clamp(min=0))
        goods = self.dropout(self.bn3(self.h2h3(goods)).clamp(min=0))
        goods = self.dropout(self.bn4(self.h2h4(goods)).clamp(min=0))
        goods = self.dropout(self.bn5(self.h2h5(goods)).clamp(min=0))
        goods = self.dropout(self.bn6(self.h2h6(goods)).clamp(min=0))
        goods = self.dropout(self.bn7(self.h2h7(goods)).clamp(min=0))
        
        output = self.h2o(goods)
        return output

In [129]:
def train(epochs=5,lr=0.01,optimizer='Adamax'):
    print('\ntrain with optimizer {} and learning rate of {}'.format(optimizer,lr))
    train_mse_loss = []
    val_mse_loss = []
    val_rmsle_loss = []
    model = Regressor()
    loss_fn = nn.MSELoss()
    if optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(),lr=lr)
    elif optimizer == 'Adamax':
        optimizer = optim.Adamax(model.parameters(),lr=lr)
    elif optimizer == 'Adadelta':
        optimizer = optim.Adadelta(model.parameters(),lr=lr)
    elif optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(),lr=lr)
    elif optimizer == 'ASGD':
        optimizer = optim.ASGD(model.parameters(),lr=lr)
    elif optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(),lr=lr)
    elif optimizer == 'Rprop':
        optimizer = optim.Rprop(model.parameters(),lr=lr)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(),lr=lr,momentum=0.8)
    data = batched_train_data
    for epoch in range(epochs):
        start = time()
        val_mse = []
        val_rmsle = []
        train_loss = []
        for i,(x,y) in enumerate(data):
            inputs = get_xv_dict(x)
            outputs = Variable(torch.FloatTensor(y))
            preds = model(inputs)
            loss = loss_fn(preds,outputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            def validate_model(x,y):
                ypreds = model(x).detach().numpy()
                yreal = y
                return rmsle(ypreds,yreal)
            def mse_(x,y):
                ypreds = model(x).detach().numpy()
                return mean_squared_error(y,ypreds)
            x_,y_ = get_valid_data()
            val_mse.append(mse_(x_,y_))
            val_rmsle.append(validate_model(x_,y_))
            train_loss.append(loss.data[0])
            if i%100 == 0:
                train_mse_loss.append(np.mean(train_loss))
                val_mse_loss.append(np.mean(val_mse))
                val_rmsle_loss.append(np.mean(val_rmsle))
                train_loss = []
                val_mae = []
                val_rmsle = []
        print('epoch {} losses are {:.4f},{:.4f},and {:.4f} this epoch takes {:.2f} minutes'.format(\
                                                    epoch,train_mse_loss[-1],\
                                                   val_mse_loss[-1],\
                                                   val_rmsle_loss[-1],(time()-start)/60))
    return model,train_mse_loss,val_mse_loss,val_rmsle_loss

In [130]:
model1,train_mae_loss1,val_mae_loss1,val_rmsle_loss1 = train(lr=0.002,optimizer='Adam')


train with optimizer Adam and learning rate of 0.002


RuntimeError: index out of range at /media/lehas-ai/新加卷/pytorch/pytorch/pytorch/aten/src/TH/generic/THTensorMath.c:287

In [3]:
train_data.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,calendar_date,day_of_week,holiday_flg,year,month,day,dayofyear,quarter
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-13,Wednesday,0,2016,1,13,13,1
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-14,Thursday,0,2016,1,14,14,1
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-15,Friday,0,2016,1,15,15,1
3,air_ba937bf13d40fb24,2016-01-16,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-16,Saturday,0,2016,1,16,16,1
4,air_ba937bf13d40fb24,2016-01-18,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-18,Monday,0,2016,1,18,18,1


In [4]:
test_data.head()

,id,visitors,air_store_id,visit_date,air_genre_name,air_area_name,calendar_date,day_of_week,holiday_flg,year,month,day,dayofyear,quarter
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-23,Sunday,0,2017,4,23,113,2
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-24,Monday,0,2017,4,24,114,2
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-25,Tuesday,0,2017,4,25,115,2
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-26,Wednesday,0,2017,4,26,116,2
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-27,Thursday,0,2017,4,27,117,2


## get the dictionaries.

In [5]:
store_set = set(train_data.air_store_id)
genre_set = set(train_data.air_genre_name)
area_set = set(train_data.air_area_name)
weekday_set = set(train_data.day_of_week)
holiday_set = set(train_data.holiday_flg)
year_set = set(train_data.year)
month_set =set(train_data.month)
day_set = set(train_data.day)
dayofyear_set = set(train_data.dayofyear)
quarter_set = set(train_data.quarter)

In [6]:
store_index_dict = {store:i for i,store in enumerate(store_set)}
genre_index_dict = {genre:i for i,genre in enumerate(genre_set)}
area_index_dict = {area:i for i,area in enumerate(area_set)}
weekday_index_dict = {weekday:i for i,weekday in enumerate(weekday_set)}
holiday_index_dict = {holi:i for i,holi in enumerate(holiday_set)}
year_index_dict = {year:i for i,year in enumerate(year_set)}
month_index_dict = {month:i for i,month in enumerate(month_set)}
day_index_dict = {day:i for i,day in enumerate(day_set)}
dayofyear_index_dict = {dayofyear:i for i,dayofyear in enumerate(dayofyear_set)}
quarter_index_dict = {quarter:i for i,quarter in enumerate(quarter_set)}

In [7]:
def store_index(data):
    return store_index_dict[data]
def genre_index(data):
    return genre_index_dict[data]
def area_index(data):
    return area_index_dict[data]
def weekday_index(data):
    return weekday_index_dict[data]
def holiday_index(data):
    return holiday_index_dict[data]
def year_index(data):
    return year_index_dict[data]
def month_index(data):
    return month_index_dict[data]
def day_index(data):
    return day_index_dict[data]
def dayofyear_index(data):
    return dayofyear_index_dict[data]
def quarter_index(data):
    return quarter_index_dict[data]

## set up data sets

In [8]:
test_data.head()

,id,visitors,air_store_id,visit_date,air_genre_name,air_area_name,calendar_date,day_of_week,holiday_flg,year,month,day,dayofyear,quarter
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-23,Sunday,0,2017,4,23,113,2
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-24,Monday,0,2017,4,24,114,2
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-25,Tuesday,0,2017,4,25,115,2
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-26,Wednesday,0,2017,4,26,116,2
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,2017-04-27,Thursday,0,2017,4,27,117,2


In [9]:
len(set(test_data.visit_date))

39

In [10]:
len(set(train_data.visit_date))

478

In [11]:
len(list(set(train_data.visit_date))[-39:])

39

## skip following lines the mask takes too much time

In [17]:
def valid_mask(data):
    if data.visit_date in list(set(train_data.visit_date))[-39:]:
        return True
    else:
        return False
def train_mask(data):
    if data.visit_date in list(set(train_data.visit_date))[-39:]:
        return False
    else:
        return True

In [15]:
train_data.iloc[:3].apply(valid_mask,axis=1)

0    False
1    False
2    False
dtype: bool

In [ ]:
data_valid = train_data[train_data.apply(valid_mask,axis=1)]

In [19]:
data_train = train_data[train_data.apply(train_mask,axis=1)]

In [18]:
data_valid.shape

(23338, 13)

In [20]:
with open('train_valid.pickle','wb') as f:
    pickle.dump((data_train,data_valid),f)

## start from here

In [32]:
with open('train_valid.pickle','rb') as f:
    data_train,data_valid = pickle.load(f)

In [33]:
data_train.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,calendar_date,day_of_week,holiday_flg,year,month,day,dayofyear,quarter
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-13,Wednesday,0,2016,1,13,13,1
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-14,Thursday,0,2016,1,14,14,1
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-15,Friday,0,2016,1,15,15,1
3,air_ba937bf13d40fb24,2016-01-16,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-16,Saturday,0,2016,1,16,16,1
4,air_ba937bf13d40fb24,2016-01-18,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-01-18,Monday,0,2016,1,18,18,1


In [34]:
data_valid.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,calendar_date,day_of_week,holiday_flg,year,month,day,dayofyear,quarter
39,air_ba937bf13d40fb24,2016-03-01,19,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-03-01,Tuesday,0,2016,3,1,61,1
40,air_ba937bf13d40fb24,2016-03-02,21,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-03-02,Wednesday,0,2016,3,2,62,1
59,air_ba937bf13d40fb24,2016-03-25,30,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-03-25,Friday,0,2016,3,25,85,1
82,air_ba937bf13d40fb24,2016-04-20,33,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-04-20,Wednesday,0,2016,4,20,111,2
83,air_ba937bf13d40fb24,2016-04-21,11,Dining bar,Tōkyō-to Minato-ku Shibakōen,2016-04-21,Thursday,0,2016,4,21,112,2


In [35]:
# normalize y
'''
y_train = pd.concat([data_train.visitors,data_valid.visitors],axis=0).values
max_log_y = np.max(np.log(y_train))
def get_y(data):
    return np.log(data)/max_log_y
'''
y_train = data_train.visitors.apply(np.log1p)
y_valid = data_valid.visitors.apply(np.log1p)

## set up data for train

In [36]:
xtrain_dict = {
    'store':data_train.air_store_id.apply(store_index).values,
    'genre':data_train.air_genre_name.apply(genre_index).values,
    'area':data_train.air_area_name.apply(area_index).values,
    'weekday':data_train.day_of_week.apply(weekday_index).values,
    'holiday':data_train.holiday_flg.apply(holiday_index).values,
    'year':data_train.year.apply(year_index).values,
    'month':data_train.month.apply(month_index).values,
    'day':data_train.day.apply(day_index).values,
    'dayofyear':data_train.dayofyear.apply(dayofyear_index).values,
    'quarter':data_train.quarter.apply(quarter_index).values
}
xval_dict = {
    'store':data_valid.air_store_id.apply(store_index).values,
    'genre':data_valid.air_genre_name.apply(genre_index).values,
    'area':data_valid.air_area_name.apply(area_index).values,
    'weekday':data_valid.day_of_week.apply(weekday_index).values,
    'holiday':data_valid.holiday_flg.apply(holiday_index).values,
    'year':data_valid.year.apply(year_index).values,
    'month':data_valid.month.apply(month_index).values,
    'day':data_valid.day.apply(day_index).values,
    'dayofyear':data_valid.dayofyear.apply(dayofyear_index).values,
    'quarter':data_valid.quarter.apply(quarter_index).values
}
xval_dict_for_val = {
    'store':Variable(torch.LongTensor(data_valid.air_store_id.apply(store_index).values),requires_grad=False),
    'genre':Variable(torch.LongTensor(data_valid.air_genre_name.apply(genre_index).values),requires_grad=False),
    'area':Variable(torch.LongTensor(data_valid.air_area_name.apply(area_index).values),requires_grad=False),
    'weekday':Variable(torch.LongTensor(data_valid.day_of_week.apply(weekday_index).values),requires_grad=False),
    'holiday':Variable(torch.LongTensor(data_valid.holiday_flg.apply(holiday_index).values),requires_grad=False),
    'year':Variable(torch.LongTensor(data_valid.year.apply(year_index).values),requires_grad=False),
    'month':Variable(torch.LongTensor(data_valid.month.apply(month_index).values),requires_grad=False),
    'day':Variable(torch.LongTensor(data_valid.day.apply(day_index).values),requires_grad=False),
    'dayofyear':Variable(torch.LongTensor(data_valid.dayofyear.apply(dayofyear_index).values),requires_grad=False),
    'quarter':Variable(torch.LongTensor(data_valid.quarter.apply(quarter_index).values),requires_grad=False)
}
xtest_dict = {
    'store':Variable(torch.LongTensor(test_data.air_store_id.apply(store_index).values),requires_grad=False),
    'genre':Variable(torch.LongTensor(test_data.air_genre_name.apply(genre_index).values),requires_grad=False),
    'area':Variable(torch.LongTensor(test_data.air_area_name.apply(area_index).values),requires_grad=False),
    'weekday':Variable(torch.LongTensor(test_data.day_of_week.apply(weekday_index).values),requires_grad=False),
    'holiday':Variable(torch.LongTensor(test_data.holiday_flg.apply(holiday_index).values),requires_grad=False),
    'year':Variable(torch.LongTensor(test_data.year.apply(year_index).values),requires_grad=False),
    'month':Variable(torch.LongTensor(test_data.month.apply(month_index).values),requires_grad=False),
    'day':Variable(torch.LongTensor(test_data.day.apply(day_index).values),requires_grad=False),
    'dayofyear':Variable(torch.LongTensor(test_data.dayofyear.apply(dayofyear_index).values),requires_grad=False),
    'quarter':Variable(torch.LongTensor(test_data.quarter.apply(quarter_index).values),requires_grad=False)
}

KeyError: 'air_cf22e368c1a71d53'

In [15]:
xval_df = pd.concat([pd.DataFrame(xval_dict),y_valid.reset_index(drop=True)],axis=1)
xtrain_df = pd.DataFrame(xtrain_dict)

In [16]:
xval_df.head()

,area,day,dayofyear,genre,holiday,month,quarter,store,weekday,year,visitors
0,88,0,60,9,0,2,0,1,1,0,2.995732
1,88,1,61,9,0,2,0,1,6,0,3.091042
2,88,24,84,9,0,2,0,1,3,0,3.433987
3,88,19,110,9,0,3,1,1,6,0,3.526361
4,88,20,111,9,0,3,1,1,0,0,2.484907


In [17]:
def get_valid_data():
    data = xval_df.sample(100)
    y = data.visitors.values
    x = {
    'store':Variable(torch.LongTensor(data.store.values),requires_grad=False),
    'genre':Variable(torch.LongTensor(data.genre.values),requires_grad=False),
    'area':Variable(torch.LongTensor(data.area.values),requires_grad=False),
    'weekday':Variable(torch.LongTensor(data.weekday.values),requires_grad=False),
    'holiday':Variable(torch.LongTensor(data.holiday.values),requires_grad=False),
    'year':Variable(torch.LongTensor(data.year.values),requires_grad=False),
    'month':Variable(torch.LongTensor(data.month.values),requires_grad=False),
    'day':Variable(torch.LongTensor(data.day.values),requires_grad=False),
    'dayofyear':Variable(torch.LongTensor(data.dayofyear.values),requires_grad=False),
    'quarter':Variable(torch.LongTensor(data.quarter.values),requires_grad=False)
    }
    return x,y

In [18]:
def batch_up(batch_size,features,labels):
    assert len(features) == len(labels)
    output_batch = []
    
    sample_size = len(features)
    for start_i in range(0,sample_size,batch_size):
        end_i = start_i + batch_size
        batch_feature = features[start_i:end_i]
        batch_label = labels[start_i:end_i]
        output_batch.append((batch_feature,batch_label))
        
    return output_batch

batched_train_data = batch_up(128,xtrain_df,y_train.values)

In [19]:
len(batched_train_data)

1788

In [20]:
def get_x_dict(data):
    diction = {
    'store':Variable(torch.LongTensor(data.store.values)),
    'genre':Variable(torch.LongTensor(data.genre.values)),
    'area':Variable(torch.LongTensor(data.area.values)),
    'weekday':Variable(torch.LongTensor(data.weekday.values)),
    'holiday':Variable(torch.LongTensor(data.holiday.values)),
    'year':Variable(torch.LongTensor(data.year.values)),
    'month':Variable(torch.LongTensor(data.month.values)),
    'day':Variable(torch.LongTensor(data.day.values)),
    'dayofyear':Variable(torch.LongTensor(data.dayofyear.values)),
    'quarter':Variable(torch.LongTensor(data.quarter.values))
    }
    return diction

In [21]:
def retrive_y(data):
    return np.exp(data)-1
def rmsle(preds,real):
    return np.sqrt(mean_squared_error(real,preds))
def valid_full(model):
    ypreds = model(xval_dict_for_val).detach().numpy()
    #ypreds = pd.Series(ypreds.reshape((-1,))).apply(retrive_y).values
    yreal = xval_df.visitors.values
    return rmsle(ypreds,yreal)

## define network

In [22]:
print(len(store_set),len(genre_set),len(area_set),len(weekday_set),len(holiday_set),len(year_set),\
     len(month_set),len(day_set),len(dayofyear_set),len(quarter_set))

829 14 103 7 2 2 12 31 366 4


In [23]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor,self).__init__()
        
        self.store_embeddor = nn.Embedding(len(store_set),15)
        self.genre_embeddor = nn.Embedding(len(genre_set),6)
        self.area_embeddor = nn.Embedding(len(area_set),10)
        self.weekday_embeddor = nn.Embedding(len(weekday_set),3)
        self.holiday_embeddor = nn.Embedding(len(holiday_set),2)
        self.year_embeddor = nn.Embedding(len(year_set),2)
        self.month_embeddor = nn.Embedding(len(month_set),6)
        self.day_embeddor = nn.Embedding(len(day_set),8)
        self.dayofyear_embeddor = nn.Embedding(len(dayofyear_set),10)
        self.quarter_embeddor = nn.Embedding(len(quarter_set),2)
        
        self.e2h = nn.Linear(64,2000)
        self.bn0 = nn.BatchNorm1d(2000)
        self.h2h1 = nn.Linear(2000,1600)
        self.bn1 = nn.BatchNorm1d(1600)
        self.h2h2 = nn.Linear(1600,1300)
        self.bn2 = nn.BatchNorm1d(1300)
        self.h2h3 = nn.Linear(1300,1000)
        self.bn3 = nn.BatchNorm1d(1000)
        self.h2h4 = nn.Linear(1000,680)
        self.bn4 = nn.BatchNorm1d(680)
        self.h2h5 = nn.Linear(680,380)
        self.bn5 = nn.BatchNorm1d(380)
        self.h2h6 = nn.Linear(380,200)
        self.bn6 = nn.BatchNorm1d(200)
        self.h2h7 = nn.Linear(200,80)
        self.bn7 = nn.BatchNorm1d(80)
        self.h2o = nn.Linear(80,1)
        
        self.dropout = nn.Dropout(0.1)
        
        for layer in [self.e2h,self.h2h1,self.h2h2,self.h2h3,self.h2h4,self.h2h5,self.h2h6,self.h2h7,self.h2o]:
            self.init_wb(layer)
            
    def init_wb(self,layer):
        init.normal(layer.weight,mean=0.,std=0.1)
        init.constant(layer.bias,0)
        
    def forward(self,inputs):
        store_embeds = self.store_embeddor(inputs['store']).view(-1,15)
        genre_embeds = self.genre_embeddor(inputs['genre']).view(-1,6)
        area_embeds = self.area_embeddor(inputs['area']).view(-1,10)
        weekday_embeds = self.weekday_embeddor(inputs['weekday']).view(-1,3)
        holiday_embeds = self.holiday_embeddor(inputs['holiday']).view(-1,2)
        year_embeds = self.year_embeddor(inputs['year']).view(-1,2)
        month_embeds = self.month_embeddor(inputs['month']).view(-1,6)
        day_embeds = self.day_embeddor(inputs['day']).view(-1,8)
        dayofyear_embeds = self.dayofyear_embeddor(inputs['dayofyear']).view(-1,10)
        quarter_embeds = self.quarter_embeddor(inputs['quarter']).view(-1,2)
        
        merged = torch.cat([store_embeds,genre_embeds,area_embeds,weekday_embeds,\
                        holiday_embeds,year_embeds,month_embeds,day_embeds,\
                        dayofyear_embeds,quarter_embeds],1)
        goods = self.dropout(self.bn0(self.e2h(merged)).clamp(min=0))
        goods = self.dropout(self.bn1(self.h2h1(goods)).clamp(min=0))
        goods = self.dropout(self.bn2(self.h2h2(goods)).clamp(min=0))
        goods = self.dropout(self.bn3(self.h2h3(goods)).clamp(min=0))
        goods = self.dropout(self.bn4(self.h2h4(goods)).clamp(min=0))
        goods = self.dropout(self.bn5(self.h2h5(goods)).clamp(min=0))
        goods = self.dropout(self.bn6(self.h2h6(goods)).clamp(min=0))
        goods = self.dropout(self.bn7(self.h2h7(goods)).clamp(min=0))
        
        output = self.h2o(goods)
        return output

In [24]:
def train(epochs=5,lr=0.01,optimizer='Adamax'):
    print('\ntrain with optimizer {} and learning rate of {}'.format(optimizer,lr))
    train_mse_loss = []
    val_mse_loss = []
    val_rmsle_loss = []
    model = Regressor()
    loss_fn = nn.MSELoss()
    if optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(),lr=lr)
    elif optimizer == 'Adamax':
        optimizer = optim.Adamax(model.parameters(),lr=lr)
    elif optimizer == 'Adadelta':
        optimizer = optim.Adadelta(model.parameters(),lr=lr)
    elif optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(),lr=lr)
    elif optimizer == 'ASGD':
        optimizer = optim.ASGD(model.parameters(),lr=lr)
    elif optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(),lr=lr)
    elif optimizer == 'Rprop':
        optimizer = optim.Rprop(model.parameters(),lr=lr)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(),lr=lr,momentum=0.8)
    data = batched_train_data
    for epoch in range(epochs):
        start = time()
        val_mse = []
        val_rmsle = []
        train_loss = []
        for i,(x,y) in enumerate(data):
            inputs = get_x_dict(x)
            outputs = Variable(torch.FloatTensor(y))
            preds = model(inputs)
            loss = loss_fn(preds,outputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            def validate_model(x,y):
                ypreds = model(x).detach().numpy()
                yreal = y
                return rmsle(ypreds,yreal)
            def mse_(x,y):
                ypreds = model(x).detach().numpy()
                return mean_squared_error(y,ypreds)
            x_,y_ = get_valid_data()
            val_mse.append(mse_(x_,y_))
            val_rmsle.append(validate_model(x_,y_))
            train_loss.append(loss.data[0])
            if i%100 == 0:
                train_mse_loss.append(np.mean(train_loss))
                val_mse_loss.append(np.mean(val_mse))
                val_rmsle_loss.append(np.mean(val_rmsle))
                train_loss = []
                val_mae = []
                val_rmsle = []
        print('epoch {} losses are {:.4f},{:.4f},and {:.4f} this epoch takes {:.2f} minutes'.format(\
                                                    epoch,train_mse_loss[-1],\
                                                   val_mse_loss[-1],\
                                                   val_rmsle_loss[-1],(time()-start)/60))
    return model,train_mse_loss,val_mse_loss,val_rmsle_loss

In [25]:
model1,train_mae_loss1,val_mae_loss1,val_rmsle_loss1 = train(epochs=15,lr=0.001,optimizer='Adam')


train with optimizer Adam and learning rate of 0.001
epoch 0 losses are 0.6926,0.8342,and 0.8332 this epoch takes 5.02 minutes
epoch 1 losses are 0.6810,0.6738,and 0.8175 this epoch takes 4.92 minutes
epoch 2 losses are 0.6678,0.6517,and 0.8168 this epoch takes 5.13 minutes
epoch 3 losses are 0.6625,0.6329,and 0.7843 this epoch takes 5.72 minutes
epoch 4 losses are 0.6505,0.6178,and 0.7932 this epoch takes 5.88 minutes
epoch 5 losses are 0.6390,0.6076,and 0.7768 this epoch takes 5.78 minutes
epoch 6 losses are 0.6305,0.6070,and 0.7811 this epoch takes 6.29 minutes
epoch 7 losses are 0.6234,0.6000,and 0.7810 this epoch takes 6.11 minutes
epoch 8 losses are 0.6107,0.6018,and 0.7861 this epoch takes 6.21 minutes
epoch 9 losses are 0.6036,0.6068,and 0.7921 this epoch takes 6.33 minutes
epoch 10 losses are 0.5940,0.6118,and 0.7916 this epoch takes 5.46 minutes
epoch 11 losses are 0.5873,0.6087,and 0.7901 this epoch takes 8.14 minutes
epoch 12 losses are 0.5696,0.6009,and 0.7862 this epoch 

In [29]:
valid_full(model1)

2.3446299534092763